In [1]:
# 0. Setup
import os
import importlib
import numpy as np
import pandas as pd

import tdabball as tdb
importlib.reload(tdb)

print("Working directory:", os.getcwd())
os.listdir()

Working directory: /home/jovyan/work/tdaMCBBproject


['README.md',
 '.git',
 'tdabball.py',
 'TDAbasketballNotebook.ipynb',
 'MCBBdata.Rproj',
 'downloadshotdata.qmd',
 '.DS_Store',
 '.ipynb_checkpoints',
 'data',
 '__pycache__']

In [2]:
shots = tdb.load_shot_data("data")   # loads all NBA + MBB parquet files
nba = shots.nba_by_season            # dict: {'2012': df, '2013': df, ...}
mbb = shots.mbb_by_season

In [3]:
# Define eras
era1_years = ["2012", "2013", "2014"]
era2_years = ["2015", "2016", "2017", "2018"]
era3_years = ["2019", "2020", "2021", "2022", "2023", "2024"]

In [4]:
X_era1 = tdb.era_cloud(nba, era1_years)
X_era2 = tdb.era_cloud(nba, era2_years)
X_era3 = tdb.era_cloud(nba, era3_years)

X_era1.shape, X_era2.shape, X_era3.shape

((775703, 2), (1125107, 2), (1684604, 2))

In [5]:
def subsample(X, n=15000, seed=0):
    rng = np.random.default_rng(seed)
    if len(X) <= n:
        return X
    idx = rng.choice(len(X), size=n, replace=False)
    return X[idx]

In [8]:
X1 = subsample(X_era1, n=5000, seed=1)
X2 = subsample(X_era2, n=5000, seed=2)
X3 = subsample(X_era3, n=5000, seed=3)

X1.shape, X2.shape, X3.shape   # all should be (15000, 2)

((5000, 2), (5000, 2), (5000, 2))

In [9]:
dgms_era1 = tdb.persistence_gtda(X1, maxdim=1)

In [10]:
dgms_era2 = tdb.persistence_gtda(X2, maxdim=1)

In [11]:
dgms_era3 = tdb.persistence_gtda(X3, maxdim=1)

In [12]:
[d.shape for d in dgms_era1], [d.shape for d in dgms_era2], [d.shape for d in dgms_era3]

([(3,), (3,)], [(3,), (3,)], [(3,), (3,)])